In [1]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2.5.0
2.5.1


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
SAMPLE = 1000000
EPOCHS = 20

# Florida voter
df = pd.read_csv('../dataverse_files/fl_voter_reg/fl_reg_name_race.csv.gz')
df.dropna(subset=['name_first', 'name_last'], inplace=True)
df['race'] = df.race.map({'native_indian': 'other', 'asian': 'asian', 'nh_black': 'nh_black', 'hispanic': 'hispanic', 'nh_white': 'nh_white', 'other': 'other', 'multi_racial': 'other', 'unknown': 'unknown'})
df

,name_first,name_last,race
0,Walker,Elizabeth,nh_white
1,Palmer,Alton,nh_white
2,Mc Cleod,Alicia,nh_black
3,Scarborough,Dale,nh_white
4,Walker,Daniel,nh_white
...,...,...,...
13653889,Walters,William,nh_white
13653890,Sawyer,Matthew,nh_white
13653891,Thomas,Janine,nh_white
13653892,Campbell,Angel,other


In [3]:
sdf = df[df.race.isin(['unknown']) == False].groupby(['race']).sample(int(SAMPLE/5), random_state=21)
del df

# Additional features
sdf['name_first'] = sdf.name_first.str.title()
sdf['name_last'] = sdf.name_last.str.title()
sdf

,name_first,name_last,race
779856,Kelley,Mi,asian
10722411,Pitman,Ariel,asian
9871739,Patel,Ranjanbala,asian
6278811,Inch,Corazon,asian
6085057,Mathew,Lilly,asian
...,...,...,...
2308712,Castro,Gisele,other
5945501,Brooks,Dyron,other
8160933,Carson,Mary,other
8397639,Amer,Ossama,other


In [4]:
rdf = sdf.groupby('race').agg({'name_last': 'count'})
rdf.to_csv('../dataverse_files/fl_voter_reg/lstm/fl_name_five_cat_race.csv', columns=[])
rdf

,name_last
race,
asian,200000
nh_black,200000
nh_white,200000
other,200000


In [5]:
sdf.groupby('race').agg({'name_last': 'nunique'})

,name_last
race,
asian,49116
nh_black,43038
nh_white,12241
other,35977


## Preprocessing the input data

In [6]:
# concat last name and first name
sdf['name_last_name_first'] = sdf['name_last'] + ' ' + sdf['name_first']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 1313
Max feature len = 44, Avg. feature len = 12


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [7]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

640000 train sequences
160000 test sequences
Pad sequences (samples x time)
X_train shape: (640000, 20)
X_test shape: (160000, 20)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (640000, 4)
y_test shape: (160000, 4)


In [8]:
print('Build model...')

input_ = keras.layers.Input(shape=(feature_len))
layer_ = Embedding(num_words, 32) (input_)
layer_ = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(layer_, training=True)
output = Dense(num_classes, activation='softmax') (layer_)

model = keras.models.Model(inputs=input_, outputs=output)

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 20, 32)            42016     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               82432     
_________________________________________________________________
dense (Dense)                (None, 4)                 516       
Total params: 124,964
Trainable params: 124,964
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=1)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/20
18000/18000 [==============================] - 393s 20ms/step - loss: 1.1300 - accuracy: 0.4909 - val_loss: 1.0553 - val_accuracy: 0.5357
Epoch 2/20
18000/18000 [==============================] - 376s 21ms/step - loss: 1.0413 - accuracy: 0.5462 - val_loss: 1.0253 - val_accuracy: 0.5538
Epoch 3/20
18000/18000 [==============================] - 345s 19ms/step - loss: 1.0134 - accuracy: 0.5613 - val_loss: 1.0120 - val_accuracy: 0.5621
Epoch 4/20
18000/18000 [==============================] - 338s 19ms/step - loss: 0.9968 - accuracy: 0.5697 - val_loss: 1.0029 - val_accuracy: 0.5675
Epoch 5/20
18000/18000 [==============================] - 333s 18ms/step - loss: 0.9871 - accuracy: 0.5749 - val_loss: 0.9989 - val_accuracy: 0.5699
Epoch 6/20
18000/18000 [==============================] - 332s 18ms/step - loss: 0.9802 - accuracy: 0.5806 - val_loss: 0.9943 - val_accuracy: 0.5716
Epoch 7/20
18000/18000 [==============================] - 332s 18ms/step - loss: 0.9748 - accurac

In [10]:
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 0.9844765663146973
Test accuracy: 0.5787374973297119


## Confusion Matrix

In [11]:
p = model.predict(X_test, verbose=2) # to predict probability
y_pred = np.argmax(p, axis=-1)
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

5000/5000 - 23s
              precision    recall  f1-score   support

       asian       0.74      0.70      0.72     40000
    nh_black       0.59      0.72      0.64     40000
    nh_white       0.51      0.70      0.59     40000
       other       0.45      0.21      0.28     40000

    accuracy                           0.58    160000
   macro avg       0.57      0.58      0.56    160000
weighted avg       0.57      0.58      0.56    160000

[[27887  2832  4627  4654]
 [ 1071 28673  8161  2095]
 [ 1462  7264 27815  3459]
 [ 7323 10176 14266  8235]]


## Save model

In [12]:
model.save('../models/fl_voter_reg/lstm/fl_all_fullname_lstm_5_cat.h5')

In [13]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('../dataverse_files/fl_voter_reg/lstm/fl_all_fullname_vocab_5_cat.csv', index=False, encoding='utf-8')